Grad-CAM applied to ResNet-18 image dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#import zipfile
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Define your dataset path
FEMUR_cesarean_train = "/content/drive/MyDrive/TESE/Project/FEMUR_cesarean_train_cv1_layer4"
FEMUR_cesarean_test = "/content/drive/MyDrive/TESE/Project/FEMUR_cesarean_test_cv1_layer4"
FEMUR_vaginal_train = "/content/drive/MyDrive/TESE/Project/FEMUR_vaginal_train_cv1_layer4"
FEMUR_vaginal_test = "/content/drive/MyDrive/TESE/Project/FEMUR_vaginal_test_cv1_layer4"

HEAD_cesarean_train = "/content/drive/MyDrive/TESE/Project/HEAD_cesarean_train_cv1_layer4"
HEAD_cesarean_test = "/content/drive/MyDrive/TESE/Project/HEAD_cesarean_test_cv1_layer4"
HEAD_vaginal_train = "/content/drive/MyDrive/TESE/Project/HEAD_vaginal_train_cv1_layer4"
HEAD_vaginal_test = "/content/drive/MyDrive/TESE/Project/HEAD_vaginal_test_cv1_layer4"

ABDOMEN_cesarean_train = "/content/drive/MyDrive/TESE/Project/ABDOMEN_cesarean_train_cv1_layer4"
ABDOMEN_cesarean_test = "/content/drive/MyDrive/TESE/Project/ABDOMEN_cesarean_test_cv1_layer4"
ABDOMEN_vaginal_train = "/content/drive/MyDrive/TESE/Project/ABDOMEN_vaginal_train_cv1_layer4"
ABDOMEN_vaginal_test = "/content/drive/MyDrive/TESE/Project/ABDOMEN_vaginal_test_cv1_layer4"

In [3]:
# Check if directory exists
if os.path.exists(ABDOMEN_cesarean_train):
    print(f"Dataset found at {ABDOMEN_cesarean_train}")
    print("Listing files in dataset directory:\n")
    print(os.listdir(ABDOMEN_cesarean_train))  # List files and folders
else:
    print(f"Dataset not found at {ABDOMEN_cesarean_train}. Check the path!")


Dataset found at /content/drive/MyDrive/TESE/Project/ABDOMEN_cesarean_train_cv1_layer4
Listing files in dataset directory:

['PU19017940_abdomen1_557.png', 'PU18020643_abdomen1_698.png', 'PU19025956_abdomen1_719.png', 'PU17013243_abdomen1_301.png', 'PU16011929_abdomen1_258.png', 'PU9125486_abdomen1_429.png', 'PU19900168_abdomen1_82.png', 'PU20025956_abdomen1_380.png', 'PU1003661_abdomen1_6.png', 'PU9322841_abdomen1_781.png', 'PU15042514_abdomen1_665.png', 'PU18020038_abdomen1_311.png', 'PU9155326_abdomen1_242.png', 'PU12009721_abdomen1_286.png', 'PU19920963_abdomen1_358.png', 'PU9014454_abdomen1_207.png', 'PU9101141_abdomen1_756.png', 'PU19013025_abdomen1_338.png', 'PU15001131_abdomen1_487.png', 'PU19093249_abdomen1_322.png', 'PU19318732_abdomen1_355.png', 'PU19428417_abdomen1_346.png', 'PU19004518_abdomen1_334.png', 'PU9210449_abdomen1_457.png', 'PU20039248_abdomen1_386.png', 'PU12023917_abdomen1_18.png', 'PU21001673_abdomen1_392.png', 'PU99005641_abdomen1_464.png', 'PU18020432_abdome

In [ ]:
# # Define Dataset Path and Extract ZIP File ---
# zip_path = "/content/drive/MyDrive/TESE/Project/image-dataset.zip"  # Update this with your dataset location
# extract_path = "/content/selected_images/" #content is default working directory in colab
# target_folder = "dataset_images_cv_1/Abdomen_/test/Cesarean Birth/"  # Folder inside the ZIP you want to extract (modify as needed)


# # Create extraction directory if it doesnt exist
# os.makedirs(extract_path, exist_ok=True)

# # Open ZIP
# with zipfile.ZipFile(zip_path, 'r') as zip_rep: #opens zip in read mode 'r'
#   #extract target folder files
#   for file in zip_rep.namelist():
#     if file.startswith(target_folder)and file.lower().endswith(('.jpg', '.png')):
#       zip_rep.extract(file, extract_path)

# print(f"Extracted all files from '{target_folder}' to '{extract_path}'")

In [5]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, models
import torchvision.utils as vutils
import torchvision.transforms.functional as TF
from PIL import Image

In [6]:
# Define device
device = "cuda" if torch.cuda.is_available() else "cpu"

next cell: recreate architecture to match the one used for training before loading the weights

In [7]:
# Load base ResNet-18 model
model = models.resnet18(weights=None)

# Modify first convolutional layer for grayscale input
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final fully connected layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the correct device (CPU or GPU)
model.to(device)

# Load the saved model weights
model_path = "/content/drive/MyDrive/TESE/Project/best-model_cv1_abdomen_resnet18.pth"
model.load_state_dict(torch.load(model_path, map_location=device))

# Set the model to evaluation mode (important for inference)
model.eval()



ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = None
        self.gradients = None
        self.activations = None
        self.hook_layers(target_layer)

    def hook_layers(self, target_layer): # FP and BP
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0] # activations' gradients during backward [1, 512, 3, 3]
        def forward_hook(module, input, output):
            self.activations = output # layer4 activation during forward [1, 512, 3, 3]


        # Find the layer by name
        for name, module in self.model.named_modules():
            if name == target_layer:
                self.target_layer = module
                module.register_forward_hook(forward_hook)
                module.register_full_backward_hook(backward_hook)
                break

        if self.target_layer is None:
            raise ValueError(f"Layer {target_layer} not found in model.")


    def generate(self, input_tensor, class_indices=None):
        output = self.model(input_tensor) # forward pass
        if class_indices is None: # if target class is not specified, it uses class prediction (argmax)
            class_indices = torch.argmax(output, dim=1)

        self.model.zero_grad() # Clear any previous gradients
        one_hot = torch.zeros_like(output, device=device)
        for i, class_idx in enumerate(class_indices):
            one_hot[i, class_idx] = 1

        output.backward(gradient=one_hot) #
        self.model.zero_grad() # Ensure no gradients persist after Grad-CAM computation


         # Compute the Grad-CAM heatmap
        weights = torch.nn.functional.adaptive_avg_pool2d(self.gradients, 1) # avgpool - reduces spatial resolution for output_size=1 -> [1, 512, 1, 1]
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True) # cam operation - [1,1,3,3]
        cam = torch.relu(cam)

        # Normalize Grad-CAM heatmap
        cam = cam / (cam.max(dim=2, keepdim=True)[0].max(dim=3, keepdim=True)[0] + 1e-6)
        return cam.cpu().detach().numpy(), class_indices.cpu().numpy()


In [9]:
# Define Image Transformations (Ensure grayscale input and correct format)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((80, 80)),  # Match model's input size
    transforms.ToTensor(),
])

In [10]:
# === Ground Truth Extractor ===
def get_ground_truth_label(img_name, input_folder):
    if "cesarean" in input_folder.lower():
        return "Cesarean Birth"
    elif "vaginal" in input_folder.lower():
        return "Vaginal Birth"
    elif "Cesarean" in img_name:
        return "Cesarean Birth"
    elif "Vaginal" in img_name:
        return "Vaginal Birth"
    else:
        print(f"Warning: Could not determine ground truth for {img_name}. Assuming 'Cesarean Birth'.")
        return "Cesarean Birth"

In [11]:
# === Prediction Frame ===
def add_colored_frame(image, color, thickness=20):
    height, width = image.shape[:2]
    cv2.rectangle(image, (0, 0), (width - 1, height - 1), color, thickness)
    return image

In [12]:
# === Heatmap Overlay ===
def overlay_heatmap(original_image, heatmap): #heatmap w\ size [1,3,3]
    heatmap = cv2.resize(heatmap[0], (original_image.shape[1], original_image.shape[0])) #heatmap size from [3,3] to [80,80]
    heatmap = np.uint8(255 * heatmap) #adapt to image format needed for color mapping
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) #apply color map
    original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR) # Convert grayscale to 3-channel image
    overlay = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0) #blend - 60% of grayscale image , 40% for colored heatmap
    return overlay

In [13]:
def apply_gradcam_and_save(model, input_folder, output_folder, grid_image_size=(80, 80)):
    os.makedirs(output_folder, exist_ok=True)
    grad_cam = GradCAM(model, 'layer4')

    image_list = []
    image_names = []
    original_images = []
    correct_heatmaps = []

    misclassified_tensors = []
    correctly_classified_tensors = []

    class_names = ["Cesarean Birth", "Vaginal Birth"]

    for img_name in os.listdir(input_folder):
        if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(input_folder, img_name)

        # Load image
        image = Image.open(img_path)

        # Keep a copy of the original grayscale version for visualization
        original_np = np.array(image.convert("L"))
        original_images.append(original_np)

        # Transform image to tensor: [1, 80, 80] → unsqueeze → [1, 1, 80, 80]
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Grad-CAM
        cam_heatmaps, predicted_classes = grad_cam.generate(input_tensor)
        prediction = predicted_classes[0]
        predicted_label = class_names[prediction]


        # === Ground truth from folder name ===
        ground_truth_label = get_ground_truth_label(img_name, input_folder)
        ground_truth_class = class_names.index(ground_truth_label)

        # === Choose frame color ===
        frame_color = (0, 255, 0)  # green
        if prediction != ground_truth_class:
            frame_color = (0, 0, 255)  # red

        # === Overlay and save ===
        overlay = overlay_heatmap(original_np, cam_heatmaps[0]) # cam shape = [1,3,3]
        overlay_with_frame = add_colored_frame(overlay, frame_color, thickness=20)

        save_path = os.path.join(output_folder, f"heatmap_{predicted_label}_{img_name}")
        cv2.imwrite(save_path, overlay_with_frame)
        print(f"Saved: {save_path} ; Predicted: {predicted_label}")

        # === Add to grid tensor ===
        overlay_rgb = cv2.cvtColor(overlay_with_frame, cv2.COLOR_BGR2RGB) #converts to rgb
        overlay_pil = Image.fromarray(overlay_rgb).resize(grid_image_size) # downsizes the image to fit the grid
        tensor_for_grid = TF.to_tensor(overlay_pil) #converts into a pytorch tensor [3,H,W] and normalizes pixel values

        if prediction == ground_truth_class:
            correctly_classified_tensors.append(tensor_for_grid)
            correct_heatmaps.append(cam_heatmaps[0][0])  # 2D spatial heatmap , cam shape = [3,3]
        else:
            misclassified_tensors.append(tensor_for_grid)

    # === CREATE GRID ===
    final_sorted_grid = misclassified_tensors + correctly_classified_tensors
    if final_sorted_grid:
        grid = vutils.make_grid(final_sorted_grid, nrow=20, padding=5, normalize=True)
        grid_path = os.path.join(output_folder, "grid_overlay_ordered.jpg")
        vutils.save_image(grid, grid_path)
        print(f"Grid saved at: {grid_path}")

    # === OVERLAP ONLY CORRECT HEATMAPS ===
    if correct_heatmaps: #list of NumPy arrays (raw grad-cam maps before resizing or coloring), each w\ shape [3, 3]
        mean_heatmap = np.mean(np.stack(correct_heatmaps), axis=0) # turns list of N heatmaps into 3D array w\ shape [N, 3, 3]
        mean_heatmap -= mean_heatmap.min() # subtracts the minimum value from every pixel so the lowest value is 0
        mean_heatmap /= (mean_heatmap.max() + 1e-8) # divides every pixel by the new maximum value (normalize) ; adds 1e-8 to ensure it wont be divided by 0
        heatmap_uint8 = np.uint8(255 * mean_heatmap) # Converts [0.0 – 1.0] → [0 – 255] format needed for OpenCV
        heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET) # assigns color to the values

        # Resize to match original image size
        h, w = original_images[0].shape[:2]
        heatmap_resized_blocky = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_NEAREST) #INTER_CUBIC for smoother transitions ; INTER_NEAREST for blocky look
        heatmap_resized_smooth = cv2.resize(heatmap_colored, (w, h), interpolation=cv2.INTER_CUBIC)


        mean_path_blocky = os.path.join(output_folder, "correct_mean_heatmap_blocky.jpg")
        cv2.imwrite(mean_path_blocky, heatmap_resized_blocky)
        print(f"Mean heatmap (correct only) saved at: {mean_path_blocky}")

        mean_path_smooth = os.path.join(output_folder, "correct_mean_heatmap_smooth.jpg")
        cv2.imwrite(mean_path_smooth, heatmap_resized_smooth)
        print(f"Mean heatmap (correct only) saved at: {mean_path_smooth}")


    return correct_heatmaps, original_images


In [16]:
# Define paths
input_folder = ABDOMEN_vaginal_train
output_folder = "/content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4"
# MUDAR NOME DO GRID

# Run the Grad-CAM processing and save grid
all_heatmaps, original_images = apply_gradcam_and_save(model, input_folder, output_folder)


Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4/heatmap_Vaginal Birth_PU14014150_abdomen1_240.png ; Predicted: Vaginal Birth
Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4/heatmap_Vaginal Birth_PU9158383_abdomen1_163.png ; Predicted: Vaginal Birth
Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4/heatmap_Vaginal Birth_PU4005868_abdomen1_402.png ; Predicted: Vaginal Birth
Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4/heatmap_Vaginal Birth_PU19013289_abdomen1_340.png ; Predicted: Vaginal Birth
Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4/heatmap_Vaginal Birth_PU20033640_abdomen1_839.png ; Predicted: Vaginal Birth
Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_layer4/heatmap_Vaginal Birth_PU16026210_abdomen1_63.png ; Predicted: Vaginal Birth
Saved: /content/drive/MyDrive/TESE/Project/X_ABDOMEN_vaginal_train_cv1_la